Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [73]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import math
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [25]:
batch_size = 128
num_hidden = 2048

def computeLogit(d, w0, b0, w1, b1):
  return tf.matmul(tf.nn.relu(tf.matmul(d, w0) + b0), w1) + b1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weight0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  bias0 = tf.Variable(tf.zeros([num_hidden]))

  weight1 = tf.Variable(
    tf.truncated_normal([num_hidden, num_labels]))
  bias1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = computeLogit(tf_train_dataset, weight0, bias0, weight1, bias1)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * (tf.nn.l2_loss(weight0) + tf.nn.l2_loss(weight1))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(computeLogit(tf_valid_dataset, weight0, bias0, weight1, bias1))
  test_prediction = tf.nn.softmax(computeLogit(tf_test_dataset, weight0, bias0, weight1, bias1))


In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 920.326538
Minibatch accuracy: 7.8%
Validation accuracy: 42.4%
Minibatch loss at step 500: 262.986786
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Minibatch loss at step 1000: 199.462463
Minibatch accuracy: 78.9%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 153.280441
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 113.178581
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Minibatch loss at step 2500: 88.011955
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 68.865662
Minibatch accuracy: 85.9%
Validation accuracy: 85.9%
Test accuracy: 91.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    restricted_step = step % 5
    offset = (restricted_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 673.384827
Minibatch accuracy: 14.1%
Validation accuracy: 28.5%
Minibatch loss at step 500: 246.879227
Minibatch accuracy: 99.2%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 191.187561
Minibatch accuracy: 99.2%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 148.532440
Minibatch accuracy: 99.2%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 115.579437
Minibatch accuracy: 99.2%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 90.004951
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 70.096008
Minibatch accuracy: 99.2%
Validation accuracy: 78.3%
Test accuracy: 84.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [39]:
batch_size = 128
num_hidden = 2048

def computeLogitWithDropout(d, w0, b0, w1, b1):
  return tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(d, w0) + b0), 0.5), w1) + b1

def computeLogit(d, w0, b0, w1, b1):
  return tf.matmul(tf.nn.relu(tf.matmul(d, w0) + b0), w1) + b1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weight0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  bias0 = tf.Variable(tf.zeros([num_hidden]))

  weight1 = tf.Variable(
    tf.truncated_normal([num_hidden, num_labels]))
  bias1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  #logits = computeLogit(tf_train_dataset, weight0, bias0, weight1, bias1)
  logits = computeLogitWithDropout(tf_train_dataset, weight0, bias0, weight1, bias1)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * (tf.nn.l2_loss(weight0) + tf.nn.l2_loss(weight1))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(computeLogit(tf_valid_dataset, weight0, bias0, weight1, bias1))
  test_prediction = tf.nn.softmax(computeLogit(tf_test_dataset, weight0, bias0, weight1, bias1))

In [40]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    restricted_step = step % 5
    offset = (restricted_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1085.892212
Minibatch accuracy: 5.5%
Validation accuracy: 35.8%
Minibatch loss at step 500: 251.309692
Minibatch accuracy: 98.4%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 196.225189
Minibatch accuracy: 99.2%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 150.878494
Minibatch accuracy: 99.2%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 116.992355
Minibatch accuracy: 99.2%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 91.389648
Minibatch accuracy: 97.7%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 70.626526
Minibatch accuracy: 98.4%
Validation accuracy: 78.7%
Test accuracy: 84.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [84]:
batch_size = 128
num_hidden_layer_0 = 256
num_hidden_layer_1 = 32
image_pixels = image_size * image_size

def createWeights(w, h):
    return tf.Variable(tf.truncated_normal([w, h], stddev=1.0 / math.sqrt(float(w))), name='weights')

def createBias(w):
    return tf.Variable(tf.zeros([w]), name='bias')

def buildModel():
    weights = []
    biases = []
    
    with tf.name_scope('hidden1'):
        weights.append(createWeights(image_pixels, num_hidden_layer_0))
        biases.append(createBias(num_hidden_layer_0))
        
    with tf.name_scope('hidden2'):
        weights.append(createWeights(num_hidden_layer_0, num_hidden_layer_1))
        biases.append(createBias(num_hidden_layer_1))

    with tf.name_scope('softmax_linear'):
        weights.append(createWeights(num_hidden_layer_1, num_labels))
        biases.append(createBias(num_labels))

    return weights, biases
    
def buildLogit(d, w, b):
    assert len(w) == 3
    assert len(b) == 3
    hidden1 = tf.nn.relu(tf.matmul(d, w[0]) + b[0])
    hidden2 = tf.nn.relu(tf.matmul(hidden1, w[1]) + b[1])
    return tf.matmul(hidden2, w[2]) + b[2]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Training computation.
  w, b = buildModel()
  logits = buildLogit(tf_train_dataset, w, b)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(buildLogit(tf_valid_dataset, w, b))
  test_prediction = tf.nn.softmax(buildLogit(tf_test_dataset, w, b))

In [80]:
batch_size = 128
num_hidden_1 = 512
num_hidden_2 = 128
image_pixels = image_size * image_size

def computeLogit(d, w0, b0, w1, b1, w2, b2):
  tmp = tf.matmul(tf.nn.relu(tf.matmul(d, w0) + b0), w1) + b1
  return tf.matmul(tf.nn.relu(tmp), w2) + b2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_pixels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w0 = tf.Variable(tf.truncated_normal([image_pixels, num_hidden_1], stddev=1.0 / math.sqrt(float(image_pixels))))
  b0 = tf.Variable(tf.zeros([num_hidden_1]))

  w1 = tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=1.0 / math.sqrt(float(num_hidden_1))))
  b1 = tf.Variable(tf.zeros([num_hidden_2]))
  
  w2 = tf.Variable(tf.truncated_normal([num_hidden_2, num_labels], stddev=1.0 / math.sqrt(float(num_hidden_2))))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = computeLogit(tf_train_dataset, w0, b0, w1, b1, w2, b2)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #loss += 5e-4 * (tf.nn.l2_loss(w0) + tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(computeLogit(tf_valid_dataset, w0, b0, w1, b1, w2, b2))
  test_prediction = tf.nn.softmax(computeLogit(tf_test_dataset, w0, b0, w1, b1, w2, b2))

In [85]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.train.write_graph(session.graph_def, '', 'graph.pbtxt')
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.321978
Minibatch accuracy: 7.8%
Validation accuracy: 28.1%
Minibatch loss at step 500: 0.472701
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 0.466517
Minibatch accuracy: 88.3%
Validation accuracy: 87.6%
Minibatch loss at step 1500: 0.322596
Minibatch accuracy: 89.8%
Validation accuracy: 88.6%
Minibatch loss at step 2000: 0.336980
Minibatch accuracy: 88.3%
Validation accuracy: 88.8%
Minibatch loss at step 2500: 0.413911
Minibatch accuracy: 88.3%
Validation accuracy: 89.6%
Minibatch loss at step 3000: 0.485403
Minibatch accuracy: 85.9%
Validation accuracy: 89.5%
Test accuracy: 94.1%
